<a href="https://colab.research.google.com/github/iqalexmdz100/HolaMundo/blob/main/audiosdevoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Únete a nuestro servidor de Discord — AI HUB**
## ¡Descubre una comunidad llena de __modelos de artistas top__, __actualizaciones del Colab__, __soporte 24/7__, __tutoriales__ y __mucho más__!

### Además, estarás rodeado de personas apasionadas por la inteligencia artificial y la música, ¡así podrás conectarte con gente como tú y crear cosas increíbles! Dale like al video.


In [ ]:
#@title [PASO 1] Instalar dependencias (y cargar tu instalación en caché si existe para aumentar los tiempos)
# Required Libraries
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime

#@markdown Esto actualizará forzosamente las dependencias incluso después de que la instalación inicial pareciera haber funcionado.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown Esto forzará el uso de almacenamiento temporal, por lo que descargará las dependencias cada vez en lugar de en Drive. No es necesario, a menos que usted realmente quiere que 160mb de almacenamiento. (Activado por defecto para colab no de formación para aumentar la velocidad de lanzamiento inicial)
ForceTemporaryStorage = True #@param{type:"boolean"}

# Mounting Google Drive
if not ForceTemporaryStorage:
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Drive is already mounted. Proceeding...')

# Function to install dependencies with progress
def install_packages():
    packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
    pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'gradio==3.34.0',
                    'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5',
                    'numba==0.56.4', 'librosa==0.9.2', 'mega.py', 'gdown', 'onnxruntime', 'pyngrok==4.1.12']

    print("Updating and installing system packages...")
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

    print("Updating and installing pip packages...")
    subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

    print('Packages up to date.')

# Function to scan a directory and writes filenames and timestamps
def scan_and_write(base_path, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for dirpath, dirs, files in os.walk(base_path):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                try:
                    mtime = os.path.getmtime(fname)
                    writer.writerow([fname, mtime])
                except Exception as e:
                    print(f'Skipping irrelevant nonexistent file {fname}: {str(e)}')
    print(f'Finished recording filesystem timestamps to {output_file}.')

# Function to compare files
def compare_files(old_file, new_file):
    old_files = {}
    new_files = {}

    with open(old_file, 'r') as f:
        reader = csv.reader(f)
        old_files = {rows[0]:rows[1] for rows in reader}

    with open(new_file, 'r') as f:
        reader = csv.reader(f)
        new_files = {rows[0]:rows[1] for rows in reader}

    removed_files = old_files.keys() - new_files.keys()
    added_files = new_files.keys() - old_files.keys()
    unchanged_files = old_files.keys() & new_files.keys()

    changed_files = {f for f in unchanged_files if old_files[f] != new_files[f]}

    for file in removed_files:
        print(f'File has been removed: {file}')

    for file in changed_files:
        print(f'File has been updated: {file}')

    return list(added_files) + list(changed_files)

# Check if CachedRVC.tar.gz exists
if ForceTemporaryStorage:
    file_path = '/content/CachedRVC.tar.gz'
else:
    file_path = '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz'

content_file_path = '/content/CachedRVC.tar.gz'
extract_path = '/'

if not os.path.exists(file_path):
    folder_path = os.path.dirname(file_path)
    os.makedirs(folder_path, exist_ok=True)
    print('No cached dependency install found. Attempting to download GitHub backup..')

    try:
        download_url = "https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/CachedRVC.tar.gz"
        !wget -O $file_path $download_url
        print('Download completed successfully!')
    except Exception as e:
        print('Download failed:', str(e))

        # Delete the failed download file
        if os.path.exists(file_path):
            os.remove(file_path)
        print('Failed download file deleted. Continuing manual backup..')

if Path(file_path).exists():
    if ForceTemporaryStorage:
        print('Finished downloading CachedRVC.tar.gz.')
    else:
        print('CachedRVC.tar.gz found on Google Drive. Proceeding to copy and extract...')

    # Check if ForceTemporaryStorage is True and skip copying if it is
    if ForceTemporaryStorage:
         pass
    else:
        shutil.copy(file_path, content_file_path)

    print('Beginning backup copy operation...')

    with tarfile.open(content_file_path, 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join(extract_path, member.name)
            try:
                tar.extract(member, extract_path)
            except Exception as e:
                print('Failed to extract a file (this isn\'t normal)... forcing an update to compensate')
                ForceUpdateDependencies = True
        print(f'Extraction of {content_file_path} to {extract_path} completed.')

    if ForceUpdateDependencies:
        install_packages()
        ForceUpdateDependencies = False
else:
    print('CachedRVC.tar.gz not found. Proceeding to create an index of all current files...')
    scan_and_write('/usr/', '/content/usr_files.csv')

    install_packages()

    scan_and_write('/usr/', '/content/usr_files_new.csv')
    changed_files = compare_files('/content/usr_files.csv', '/content/usr_files_new.csv')

    with tarfile.open('/content/CachedRVC.tar.gz', 'w:gz') as new_tar:
        for file in changed_files:
            new_tar.add(file)
            print(f'Added to tar: {file}')

    os.makedirs('/content/drive/MyDrive/RVC_Cached', exist_ok=True)
    shutil.copy('/content/CachedRVC.tar.gz', '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz')
    print('Updated CachedRVC.tar.gz copied to Google Drive.')
    print('Dependencies fully up to date; future runs should be faster.')

In [ ]:
#@title [PASO 2] Clonar directorio de GitHub
#%cd /content/
#!git clone https://github.com/fumiama/Retrieval-based-Voice-Conversion-WebUI
#%cd /content/Retrieval-based-Voice-Conversion-WebUI
#!mkdir -p pretrained uvr5_weights
#!git pull

import os

# Clone the latest code from the Mangio621/Mangio-RVC-Fork repository
!git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
os.chdir('/content/Mangio-RVC-Fork')
!git checkout cccdb58287f0704e89e1e218945f0caddcf478d8
os.chdir('/content/')
!mv /content/Mangio-RVC-Fork /content/Retrieval-based-Voice-Conversion-WebUI

# Forcefully delete any existing torchcrepe dependency from an earlier run
!rm -rf /Retrieval-based-Voice-Conversion-WebUI/torchcrepe

# Download the torchcrepe folder from the maxrmorrison/torchcrepe repository
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Retrieval-based-Voice-Conversion-WebUI/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder

# Change the current directory to /content/Retrieval-based-Voice-Conversion-WebUI
os.chdir('/content/Retrieval-based-Voice-Conversion-WebUI')
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title [PASO 3] Descargar modelo base
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G48k.pth

#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/

In [ ]:
#@title [PASO 4] Descargar modelo (Importante)
#@markdown Introduzca la ruta URL al modelo (Mega, Drive, etc.) e inicie el código

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse

condition1 = False
condition2 = False

# condition1 here is to check if the .index was imported. 2 is for if the .pth was.

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://drive.google.com/file/d/1nlMy917ZDJH62REnsVCXpm7ghThd4TMF/view'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

if url != '':
    MODEL = ""  # Initialize MODEL variable

    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None


MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    MODEL = ""
    global condition3
    condition3 = True

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                condition1 = True
                logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                shutil.move(file_path, f'/content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth')
                condition2 = True

if condition1 is False:
    logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason


if condition2 and not condition1:
    print("Model partially imported! No .index file was found in the model download. The author may have forgotten to add the index file.")
elif condition1 and condition2:
    print("Model successfully imported!")
elif condition3:
    pass  # Do nothing when condition3 is true
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

!rm -r /content/unzips/
!rm -r /content/zips/

Downloading...
From (uriginal): https://drive.google.com/uc?id=1nlMy917ZDJH62REnsVCXpm7ghThd4TMF
From (redirected): https://drive.google.com/uc?id=1nlMy917ZDJH62REnsVCXpm7ghThd4TMF&confirm=t&uuid=3a59540e-93fd-4ecc-bc7f-9b609af80707
To: /content/zips/1nlMy917ZDJH62REnsVCXpm7ghThd4TMF.zip
100% 225M/225M [00:01<00:00, 209MB/s]
Found .pth file: LITkillah.pth
Found index file: added_IVF1380_Flat_nprobe_1_LITkillah_v2.index
Model successfully imported!


In [ ]:
#@title [PASO FINAL] Iniciar interfaz (Haz click en el enlace que se generará)
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!python3 infer-web.py --colab --pycmd python3


/content
Use Language: en_US
2023-08-09 23:46:47.842720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 23:46:49.217385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 23:46:51 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3241cdb45f82db93b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
loading weights/LITkillah.pth
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
2023-08-09 23:47:29 | INFO | fairseq.task